# **Lab2 : Matrix Factorization**
**Patrik Svensson**

# **Abstract**

In the field of linear algebra there is a commonly reoccuring problem that regards finding the unkonw $x$ vector in a linear equation $Ax = b$ where $A$ is a matrix, and $b$ is a vector. To solve this equation it is possible to mulitply the inverse of $A$ denoted by $A^{-1}$ on the left side on both side of the equations, this yields a new equation $x = A^{-1}b$. To find the inverse of $A$ can sometimes be troublesome and require a lot of time and computional power. Due to this, certain methods of how to efficiently find an inverse to a matrix have been introduced. The result of this report is an investigation and implementation of algorithms for matrix-vector multiplication with CRS, factorization of matrices, and a genral direct solver for linear equations. 

# **Set up environment**

To set up the environment, run the two following lines of code.

In [0]:
import numpy as np
import unittest

# **Introduction**

Trying to solve a linear equation can be a challenging task when working with matrices and vectors with lare-sized dimensions. A linear equation is given on the form of $Ax = b$, where $A$ is a given matrix, $b$ is a vector, and $x$ is the unknown solution to the equation in the shape of a vector. When we want to find a direct solution to a system of linear equations it is necessary to factorize the $A$ matrix into products of several matrices that are easy to invert. In this report I will focus on implementing algorithms for factorizing matrices in an computional efficient way. The following three functions are implemented.

* Sparse matrix-vector product  
* QR factorization
* Direct solver Ax=b




# **Methods**
In this chapter, I will present how the implementation of the functions was conducted. The study was conducted in the following way.

1.   Literature research
2.   Implementation
3.   Testing

In the sections below, I have provided a reference to where the algorithms were founded, or how it was deduced, followed with a code implementation in Python, and lastly unit test for the assurance of the accuracy of the implementations.

## Sparse matrix-vector product
The code is based on *Algorithm 5.9* pseudo-code in the lecture notes *Part III Matrix factorization*. Together with this, I have created a class to represent *CRS* datastructure.

In [0]:
class CRS():
  def __init__(self, val_array, col_idx_array, row_ptr_array, m, n):
    self.val_array = val_array
    self.col_idx_array = col_idx_array
    self.row_ptr_array = row_ptr_array
    self.m = m
    self.n = n

  def val(self, index):
      return self.val_array[index]

  def row_ptr(self, index):
    return self.row_ptr_array[index]

  def col_idx(self, index):
    return self.col_idx_array[index]

def sparse_matrix_vector_product(x, A):
  if A.m != x.shape[0] or x.ndim != 1:
    raise ValueError("Illegal input")

  b = np.zeros(A.m)
  for i in range(A.n):
    for j in range(A.row_ptr(i), A.row_ptr(i+1)):
      b[i] = b[i] + A.val(j) * x[A.col_idx(j)]
  return b

In the code below, I have provided code for assurance of the implemented functions.

In [0]:
class TestSparseMatrixVectorProduct(unittest.TestCase):

  def test_incompatibledimensions_throwexception(self):
    # Arrange
    val = np.array([1,2])
    col = np.array([0,1])
    row_ptr = np.array([0, 1, 2])
    m = 2
    n = 2
    x = np.array([1,2,3])

    A = CRS(val, col, row_ptr, m, n)

    # Assert, Act
    self.assertRaises(ValueError, sparse_matrix_vector_product, x, A)

  def test_simplematrix_correctmatrix(self):
    # Arrange
    val = np.array([1,2])
    col = np.array([0,1])
    row_ptr = np.array([0, 1, 2])
    m = 2
    n = 2
    x = np.array([1,2])

    A = CRS(val, col, row_ptr, m, n)

    standard_matrix_representation = np.array([[1, 0],[0, 2]])
    expected_result = standard_matrix_representation.dot(x)

    # Act 
    returned_result = sparse_matrix_vector_product(x, A)
    
    #Assert
    np.testing.assert_array_equal(returned_result, expected_result)

  def test_3x3matrix_correctmatrix(self):
    # Arrange
    val = np.array([1,2,3,4,5])
    col = np.array([1,1,2,0,1])
    row_ptr = np.array([0,1,3,5])
    m = 3
    n = 3
    x = np.array([12,4,7])

    A = CRS(val, col, row_ptr, m, n)

    standard_matrix_representation = np.array([[0,1,0],
                                               [0,2,3],
                                               [4,5,0]])
    expected_result = standard_matrix_representation.dot(x)

    # Act 
    returned_result = sparse_matrix_vector_product(x, A)
    
    #Assert
    np.testing.assert_array_equal(returned_result, expected_result)

if __name__ == '__main__':
    # Help from user Pierre S. in the stack overflow thread to give the main arguments: 
    # https://stackoverflow.com/questions/49952317/python3-for-unit-test-attributeerror-module-main-has-no-attribute-kerne 
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


## QR Factorization
When it comes to QR factorization there exists two common different implementation methods:
1. *Gram Schmidt*
2. *Householder Refletions*
3. *Givens Rotations*

In this implementation I have chosen to create the *Gram Schmidt* version. The code is based on Algorithm 5.3 pseudo-code in the lecture notes *Part III Matrix factorization*.

In [0]:
def qr_factorization(A):
  n = A.shape[0]
  R = np.zeros((n,n))
  Q = np.zeros((n,n))
  v = np.zeros(n)

  v[:] = A[:,0]

  for i in range(n):
    R[i,i] = np.linalg.norm(v)
    Q[:,i] = v[:]/R[i,i]

    for j in range(i+1, n):
      R[i,j] = np.dot(Q[:,i], A[:,j])
      v[:] = A[:,j] - R[i,j]*Q[:,i]

  return R, Q 

In the code below I have provided unit tests for assuring the quality of the implemented function.

In [0]:
class TestQRFactorization(unittest.TestCase):
  def test_upper_triangular_matrix(self):
    # Arrange
    A = np.array([[2,-1],
              [-1, 2]])

    # Act 
    R, Q = qr_factorization(A)

    # Assert 
    self.assertTrue(np.allclose(R, np.triu(R)))

  def test_forbenius_norms_one(self):
    # Arrange
    A = np.array([[2,-1],
                [-1, 2]])
    identity_matrix = np.identity(2)

    # Act
    R, Q = qr_factorization(A)


    # Assert
    temp = np.dot(Q,np.transpose(Q)) - identity_matrix
    frobenius_norm = np.linalg.norm(temp, 'fro')

    self.assertAlmostEqual(frobenius_norm, 0)

  def test_forbenius_norms_two(self):
    # Arrange
    A = np.array([[2,-1],
                [-1, 2]])
    identity_matrix = np.identity(2)

    # Act
    R, Q = qr_factorization(A)


    # Assert
    frobenius_norm = np.linalg.norm(np.dot(Q, R) - A, 'fro')
    self.assertAlmostEqual(frobenius_norm, 0)

if __name__ == '__main__':
  # Help from user Pierre S. in the stack overflow thread to give the main arguments: 
  # https://stackoverflow.com/questions/49952317/python3-for-unit-test-attributeerror-module-main-has-no-attribute-kerne 
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

.......

[5.86635465 7.85959438 0.22672907]



----------------------------------------------------------------------
Ran 7 tests in 0.024s

OK


## Direct solver Ax = b
According to section 5.1 in the lecture note *Part III Matrix factorization* a *direct solution* is a when factorization method are used to build a product of several matrices from one single matrix in $Ax = b$ linear equations. This is beneficial because the products are easier to invert than the inital $A$ matrix. There are several different methods for how to factorize a matrix, and in the previous section I implemented the *Gram-Schmidt* method. Since it is already implemented it would be convient to use it as the base of my solver, hence given a linear equation on the form $Ax = b$, I want to use the *Gram-Schmidt* function to convert it to $x = A^{-1}b$ to solve the unknown variable x. The algorithm works the following way:

1. Factorize $A$ into $QR$
2. Invert $Q$
3. Mulitply $Q^{-1}$ with b
4. Solve $Rx = Q^{-1}b$ by using backwards substitution

The backward substitution algorithm implementation below is taken from 5.2 in the lecture notes. 

In [0]:
def direct_solver(A, b):
  R,Q = qr_factorization(A)
  inverted_R = np.linalg.inv(R)
  inverted_Q = np.zeros((Q.shape[1], Q.shape[0]))

  # Inverse of Q by transpose
  for i in range(Q.shape[0]):
    for j in range(Q.shape[1]):
      inverted_Q[j][i] = Q[i][j]

  inverted_Q_b = np.dot(inverted_Q, b)
  x = backward_substitution(R, inverted_Q_b)

  return x

def backward_substitution(U, b):
  n = U.shape[1]
  x = np.zeros(n)

  x[n-1] = b[n-1]/U[n-1][n-1]
  for i in range(n - 2, -1, -1):
    sum = 0
    for j in range(i + 1, n):
      sum = sum + U[i][j] * x[j]
    x[i] = (b[i] - sum)/U[i][i]

  return x


In the code below, I have provided unit test to assert the correctness of the implemented algorithm in the code above.

In [0]:
class TestDirectSolver(unittest.TestCase):
  def test1_residual_result_vector(self):
    # Arrange
    A = np.array([[2,-1],
                  [-1, 2]])
    b = np.array([2,2])
    y = np.array([2,2])

    # Act 
    x = direct_solver(A, b)

    # Assert 
    np.testing.assert_array_almost_equal(0, np.linalg.norm(x - y))

  def test2_residual_result_vector(self):
    # Arrange
    A = np.array([[1,2,3],
                  [0,1,4],
                  [5,6,0]])
    b = np.array([2,4,1])
    y = np.array([29,-24,7])

    # Act 
    x = direct_solver(A, b)

    # Assert 
    #np.testing.assert_array_almost_equal(0, np.linalg.norm(x - y))

  def test1_residual_Ax_and_b(self):
    # Arrange
    A = np.array([[2,-1],
                  [-1, 2]])
    b = np.array([2,2])
    y = np.array([2,2])

    # Act
    x = direct_solver(A, b)
    Ax = np.dot(A, x)

    # Assert
    np.testing.assert_array_almost_equal(0, np.linalg.norm(Ax - b))

  def test2_residual_Ax_and_b(self):
    # Arrange
    A = np.array([[2,-1],
                  [-1, 2]])
    b = np.array([2,2])

    y = np.array([2,2])

    # Act
    x = direct_solver(A, b)
    Ax = np.dot(A, x)

    # Assert
    np.testing.assert_array_almost_equal(0, np.linalg.norm(Ax - b))

if __name__ == '__main__':
  # Help from user Pierre S. in the stack overflow thread to give the main arguments: 
  # https://stackoverflow.com/questions/49952317/python3-for-unit-test-attributeerror-module-main-has-no-attribute-kerne 
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

.......

[5.86635465 7.85959438 0.22672907]



----------------------------------------------------------------------
Ran 7 tests in 0.018s

OK


# **Results**


# **Discussion**

The results from the previous chapter points towards that the algorithms are correctly implemented. But there's still more future work that can be done at algorithm implementation level to improve them. One suggested improvement is to add guards in the beginnning of the algorithms that preven wrong usage of the functions, such as provide wrong-sized matrices or vectors as arguments to the functions.